In [27]:
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle 

In [28]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [29]:
# Preprocess the data 
data = data.drop(['RowNumber','CustomerId','Surname'],axis = 1)

In [30]:
# Encoding categorical variables 
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])


In [31]:
## One hot encode the Geography column
from sklearn.preprocessing import OneHotEncoder 

OneHotEncoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoder = OneHotEncoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoder,columns=OneHotEncoder_geo.get_feature_names_out(['Geography']))

In [32]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [33]:
# combine one hot encoded columns with the original data 
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [34]:
# Split the data into features and target 

x = data.drop('EstimatedSalary',axis=1)
y = data['EstimatedSalary']

In [35]:
with open('label_encoder_gender_reg.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl_reg','wb') as file:
    pickle.dump(OneHotEncoder_geo,file)
    

In [36]:
## Split the data into training and testing data
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [37]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


In [38]:
with open('scaler_reg.pkl','wb') as file:
    pickle.dump(scaler,file)    

In [39]:
## ANN With regressioon problem statement 

import tensorflow as tf 

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense




In [40]:
# Build the model 

model = Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)), ## Hidden layer connected with input layer
    Dense(32,activation= 'relu'),
    Dense(1)
])


# compile the model 

model.compile(optimizer = 'adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()

c:\Users\HP\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
## Train this model 
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard 
import datetime

# Set up Tensor board 

log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir = log_dir,histogram_freq = 1)

In [42]:
# setup Early Stopping 
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
 

In [43]:
# Train the model 

history = model.fit(
    x_train,y_train,
    validation_data = (x_test,y_test),
    epochs = 100,
    callbacks = [early_stopping_callback,tensorboard_callback]
)

Epoch 1/100


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 99529.2812 - mae: 99529.2812 - val_loss: 99537.4844 - val_mae: 99537.4844
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 98753.9375 - mae: 98753.9375 - val_loss: 97844.2812 - val_mae: 97844.2812
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 97438.8750 - mae: 97438.8750 - val_loss: 93470.7891 - val_mae: 93470.7891
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 92250.5234 - mae: 92250.5234 - val_loss: 86265.0078 - val_mae: 86265.0078
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 84799.5234 - mae: 84799.5234 - val_loss: 77153.0469 - val_mae: 77153.0469
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 74860.0547 - mae: 74860.0547 - val_loss: 67955.3438 - val_mae: 67955.3438
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 66176.4688 - mae: 66176.4688 - val_loss: 60343.8047 - val_mae: 60343.8047
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 59352.

In [44]:

%load_ext tensorboard

In [46]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 9648), started 0:00:57 ago. (Use '!kill 9648' to kill it.)

In [47]:
# Evaluate model on the test data 
test_loss,test_mae = model.evaluate(x_test,y_test)
print(f'Test MAE : {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 50210.9805 - mae: 50210.9805
Test MAE : 49912.203125


In [48]:
model.save('regression_model.keras')